In this notebook we will do a quick analysis of the surface of a sundial. We will:

- Download Sundial pointcloud data
- Load it
- Visualize the points
- Filter out a plane section
- Visualize filtered points
- Fit a Plane shape to that filtered out section
- Plot the shape together with the filtered points
- Do the same for a cone.

In [ ]:
import numpy as np
import ipyvolume as ipv
import ectopylasm as ep

# Download data

The Topoi repository has a wealth of sundial scans freely available for download (under CC BY-NC-SA 3.0 DE license). We will download the following example, either with curl (only on Unix systems) or with Python urllib:

In [ ]:
# !curl -fLo ObjID126.ply http://repository.edition-topoi.org/BSDP/ReposBSDP/BSDP0030/ObjID126.ply

In [ ]:
# import urllib.request
# import shutil

# url = "http://repository.edition-topoi.org/BSDP/ReposBSDP/BSDP0030/ObjID126.ply"
# filename = "ObjID126.ply"
# with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
#     shutil.copyfileobj(response, out_file)

# Load data

The first time we load data from PLY files, `ectopylasm` will store an optimized version of the points (vertices) from the PLY file in a new HDF5 file with a `.cache.ecto` extension. The next time the PLY file is loaded, this will increase loading time significantly. This is all done under the hood, the user doesn't have to deal with this.

In [ ]:
points = ep.pandas_vertices_from_plyfile('ObjID126.ply')

# Visualize the points

Let's see what we've got!

In this notebook we use `ipyvolume` for plotting. All the `ectopylasm` shape plotting functions work with `ipyvolume` as well. For plotting pointclouds, one could also use `pptk`, which has a higher framerate, but is not integrated into the notebook, and doesn't support plotting shape surfaces.

In [ ]:
ipv.clear()
ipv.scatter(points.x, points.y, points.z, marker='circle_2d', size=0.2)
ipv.show()

# Filter out a plane section

The bottom front part of the sundial seems like it's planar. Let's try to isolate that part and fit it to an actual plane.

In [ ]:
# estimate the parameters of the plane that encompasses our region
plane_point = (0, -70, -200)
plane_normal = (0, -1, 1)

In [ ]:
plane = ep.Plane.from_point(*plane_normal, plane_point)

In [ ]:
ipv.clear()
ipv.scatter(points.x, points.y, points.z, marker='circle_2d', size=0.2)
ep.plot_plane(plane)
ipv.show()

That's not really it yet, let's adjust a bit.

In [ ]:
# tweak the parameters of the plane until the result looks good enough for filtering
plane_point = (0, -70, -200)
plane_normal = (0, -1, 0.7)

plane = ep.Plane.from_point(*plane_normal, plane_point)

ipv.clear()
ipv.scatter(points.x, points.y, points.z, marker='circle_2d', size=0.2)
ep.plot_plane(plane)
ipv.show()

Looks good enough for now. Let's turn that into a filter then, shall we? We only need to estimate still the thickness. Something like 20-50 seems reasonable.

In [ ]:
filtered_points = np.array(ep.filter_points_plane(points.values.T, plane, 40)).T

In [ ]:
len(points), len(filtered_points)

In [ ]:
ipv.clear()
ipv.scatter(points.x, points.y, points.z, marker='circle_2d', size=0.2)
ipv.scatter(*filtered_points, marker='circle_2d', size=0.4, color='blue')
ipv.show()

Ok, we took in a little bit too much. Let's manually filter out the junk we don't want to fit to with some simple conditionals.

In [ ]:
condition = np.logical_and(filtered_points[0] < 50, filtered_points[0] > -70)
condition = np.logical_and(condition, filtered_points[2] < -140)
condition = np.logical_and(condition, filtered_points[2] > -220)
filtered_points_2 = filtered_points.T[condition].T

In [ ]:
ipv.clear()
ipv.scatter(points.x, points.y, points.z, marker='circle_2d', size=0.2)
ipv.scatter(*filtered_points_2, marker='circle_2d', size=0.4, color='blue')
ipv.show()

That's a nice planar sample.

# Fit a plane

Let's fit a plane to this section to find its parameters.

In [ ]:
fit_result = ep.fit_plane(filtered_points_2)

# Visualize results

Finally, let's see what we've got!

First we print the parameters, then we inspect the fit compared to the filtered points visually.

In [ ]:
print(fit_result)

In [ ]:
ipv.clear()
ipv.scatter(*filtered_points_2, marker='circle_2d', size=0.4, color='blue')
ep.plot_plane_fit(fit_result)
ipv.show()

As we can see, the fit is really good. We can use the plane parameters to do further analysis.

# Same for cone

The top part of the structure actually looks like some kind of conal section. Could we fit a cone to this part? Let's try!

For the filtering, we're just going to start with a rough coordinate slice, because guessing the cone parameters will be hard. The apex will be somewhere outside of the space.

In [ ]:
condition = np.logical_and(points.y < -10, points.z > -100)
condition = np.logical_and(condition, points.z < -20)
condition = np.logical_and(condition, points.x < 65)
condition = np.logical_and(condition, points.x > -85)
cone_filtered_points = points[condition]

In [ ]:
ipv.clear()
ipv.scatter(*cone_filtered_points.values.T, marker='circle_2d', size=0.2)
ipv.show()

Now, fitting this naively will take a very long time. It makes sense to provide some initial guesses to help the fitter along.

In [ ]:
# don't just run naively!
# fit_cone_result = ep.fit_cone(cone_filtered_points)

In [ ]:
# run with a good initial guess:
guess_cone = ep.Cone(400, 150, rot_x=np.pi, base_pos=ep.Point(0, -150, 0))

In [ ]:
ipv.clear()
ipv.scatter(*cone_filtered_points.values.T, marker='circle_2d', size=0.2)
ep.plot_cone(guess_cone)
ipv.show()

Also, for performance, let's use just a random subset of all points.

Note: `ep.random_sample` is a bit weird, it will be rewritten in a more user friendly way.

In [ ]:
cfp_dict = dict(x=cone_filtered_points.values[:,0], y=cone_filtered_points.values[:,1],
                z=cone_filtered_points.values[:,2])

In [ ]:
cone_points_sample = ep.random_sample(cfp_dict, len(cone_filtered_points),
                                      100 / len(cone_filtered_points))

In [ ]:
cone_points_sample_array = np.array((cone_points_sample['x'], cone_points_sample['y'], cone_points_sample['z']))

In [ ]:
cone_points_sample_array.shape

In [ ]:
fit_cone_result = ep.fit_cone(cone_points_sample_array, initial_guess_cone=guess_cone)

In [ ]:
ipv.clear()
ipv.scatter(*cone_filtered_points.values.T, marker='circle_2d', size=0.2)
ipv.scatter(*cone_points_sample_array, marker='circle_2d', size=0.4, color='blue')
ep.plot_cone_fit(fit_cone_result)
ipv.show()

Not sure if this makes sense, but it doesn't seem completely crazy.